# Intro to tensor flow
 - Basic models over MNIST dataset
   - Linear model
   - NN one layer node
   - Convolutional model
 - Tensoboard example
 - Save & load models

In [1]:
# Header
# Basic libraries & options
from __future__ import print_function

#Basic libraries
import numpy as np
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)
import time

# Select GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#Show images
import matplotlib.pyplot as plt
%matplotlib inline
# plt configuration
plt.rcParams['figure.figsize'] = (10, 10)        # size of images
plt.rcParams['image.interpolation'] = 'nearest'  # show exact image
plt.rcParams['image.cmap'] = 'gray'  # use grayscale 


Tensorflow version:  1.2.1


## Base keras model

In [2]:
from tensorflow.contrib.keras import datasets, layers, models, optimizers

# Data
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data(path='mnist.npz')
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.


In [3]:
# Base Keras Model
images = layers.Input(batch_shape=(None, 28, 28), dtype='float32', name='Images')

flat   = layers.Flatten(name='Flat_image')(images)
dense  = layers.Dense(500, activation='relu', name='Dense_layer')(flat)
output = layers.Dense(10, activation='softmax', name='Dense_output')(dense)

model_linear = models.Model(inputs=images, outputs=output)

# Train
sgd_optimizer = optimizers.SGD(lr=0.01)
model_linear.compile(loss='sparse_categorical_crossentropy',
                     optimizer=optimizers.SGD(lr=0.01), metrics=['accuracy'])

history_linear = model_linear.fit(X_train, y_train, batch_size=128, epochs=50,
                                  verbose=1, validation_data=(X_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 4s - loss: 1.1490 - acc: 0.7427 - val_loss: 0.6084 - val_acc: 0.8648
Epoch 2/50
60000/60000 [==============================] - 1s - loss: 0.5258 - acc: 0.8734 - val_loss: 0.4332 - val_acc: 0.8935
Epoch 3/50
60000/60000 [==============================] - 1s - loss: 0.4200 - acc: 0.8914 - val_loss: 0.3716 - val_acc: 0.9029
Epoch 4/50
60000/60000 [==============================] - 1s - loss: 0.3727 - acc: 0.9000 - val_loss: 0.3379 - val_acc: 0.9108
Epoch 5/50
60000/60000 [==============================] - 1s - loss: 0.3439 - acc: 0.9056 - val_loss: 0.3158 - val_acc: 0.9147
Epoch 6/50
60000/60000 [==============================] - 1s - loss: 0.3234 - acc: 0.9112 - val_loss: 0.2997 - val_acc: 0.9185
Epoch 7/50
60000/60000 [==============================] - 1s - loss: 0.3078 - acc: 0.9150 - val_loss: 0.2864 - val_acc: 0.9217
Epoch 8/50
60000/60000 [==============================] - 1s 

# Tensorflow control the training proccess
    - use a batch generator
    - Implement a basic training process
        - Create placeholders
        - Create a loss function
        - Create a trainer
        - Define an accuracy metric

In [4]:
# Start interactive session
sess = tf.InteractiveSession()


In [5]:
# Define input placeholders
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

# Define model using Keras layers
flat   = layers.Flatten(name='Flat_image')(x)
dense  = layers.Dense(500, activation='relu', name='Dense_layer')(flat)
output = layers.Dense(10, activation='relu', name='Dense_output')(dense)

# Define the Tensorflow Loss
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)

# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [6]:
# Define a generator to data. Same code to an HDF5 source
def batch_generator(X, y, batch_size=64):
    data_size = X.shape[0]
    # Randomize batches in each epoch
    batch_randomized = np.random.permutation(range(0, data_size-batch_size, batch_size))
    # Iterate over each batch
    for batch in batch_randomized:
        x_batch = X[batch : batch+batch_size]
        y_batch = y[batch : batch+batch_size]
        yield x_batch, y_batch


In [7]:
# Intialize vars
sess.run(tf.global_variables_initializer())


In [8]:
# Basic iterator over the epochs and the batches to execute the trainer. 
batch_size = 128
num_epoch = 50
for epoch in range(num_epoch):
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})
    print(epoch, accuracy.eval(feed_dict={x: X_test, y: y_test}))


0 0.7793
1 0.8753
2 0.8957
3 0.9063
4 0.9112
5 0.9154
6 0.9198
7 0.9217
8 0.9243
9 0.9266
10 0.9301
11 0.931
12 0.9321
13 0.933
14 0.9351
15 0.9366
16 0.9384
17 0.9392
18 0.9406
19 0.9408
20 0.9437
21 0.9427
22 0.9446
23 0.9449
24 0.9478
25 0.948
26 0.9488
27 0.9499
28 0.9515
29 0.9509
30 0.9529
31 0.9538
32 0.9544
33 0.9537
34 0.9543
35 0.9554
36 0.9549
37 0.956
38 0.9571
39 0.9574
40 0.9577
41 0.9582
42 0.9582
43 0.9587
44 0.9592
45 0.9605
46 0.9608
47 0.9606
48 0.9611
49 0.9623


In [9]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True

while stop:
    #Train
    epoch += 1
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})

    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    if epoch%10==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

Epoch:  10 Accuracy:  0.9235
Epoch:  20 Accuracy:  0.939
Epoch:  30 Accuracy:  0.9491
Epoch:  40 Accuracy:  0.9559
Epoch:  50 Accuracy:  0.96
Epoch:  60 Accuracy:  0.9647
Epoch:  70 Accuracy:  0.9664
Epoch:  80 Accuracy:  0.9689
Epoch:  90 Accuracy:  0.9704
Epoch:  100 Accuracy:  0.9722
Epoch:  110 Accuracy:  0.9736
Epoch:  120 Accuracy:  0.9747
Epoch:  130 Accuracy:  0.9753
Epoch:  140 Accuracy:  0.9763
Epoch:  150 Accuracy:  0.977
Epoch:  160 Accuracy:  0.9775
Epoch:  170 Accuracy:  0.9782
Epoch:  180 Accuracy:  0.9786
STOP. Accuracy:  0.9779


In [10]:
#When finish, close the interactive session
sess.close()

# Reset the graph to the next experiments
tf.reset_default_graph()

# Tensorflow layers and tensorboard
    - Combine Keras layers with tensorflow layers
    - Monitor layers in tensorboard

In [11]:
# Start interactive session
sess = tf.InteractiveSession()


# Convolutional model
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

image = tf.reshape(x,[-1,28,28,1])

conv1 = layers.Conv2D(20, (5,5))(image)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(20, (5,5))(pool1)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
flat = layers.Flatten(name='Flat_image')(pool2)
print('Flat layer: ', flat)


# Tensorflow Dense layer
W_dense = tf.Variable(tf.truncated_normal([320, 500], stddev=0.1), name='W_dense')
b_dense = tf.Variable(tf.constant(0.1, shape=[500]), name='b_dense')
dense1 = tf.nn.relu(tf.matmul(flat, W_dense) + b_dense)
#dense1 = layers.Dense(500, activation='relu', name='Dense_1')(flat)


output = layers.Dense(10, activation='linear', name='Dense_output')(dense1)

# Define the Tensorflow Loss
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)

# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


Flat layer:  Tensor("Flat_image/Reshape:0", shape=(?, 320), dtype=float32)


In [12]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True

while stop:
    #Train
    epoch += 1
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})

    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    if epoch%1==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

Epoch:  1 Accuracy:  0.9047
Epoch:  2 Accuracy:  0.9381
Epoch:  3 Accuracy:  0.9549
Epoch:  4 Accuracy:  0.9639
Epoch:  5 Accuracy:  0.9676
Epoch:  6 Accuracy:  0.9726
Epoch:  7 Accuracy:  0.973
Epoch:  8 Accuracy:  0.9761
Epoch:  9 Accuracy:  0.9758
Epoch:  10 Accuracy:  0.979
Epoch:  11 Accuracy:  0.9769
Epoch:  12 Accuracy:  0.9803
Epoch:  13 Accuracy:  0.9802
Epoch:  14 Accuracy:  0.9813
Epoch:  15 Accuracy:  0.9818
Epoch:  16 Accuracy:  0.9816
Epoch:  17 Accuracy:  0.9823
Epoch:  18 Accuracy:  0.9833
Epoch:  19 Accuracy:  0.9824
Epoch:  20 Accuracy:  0.984
Epoch:  21 Accuracy:  0.9828
Epoch:  22 Accuracy:  0.9847
Epoch:  23 Accuracy:  0.985
Epoch:  24 Accuracy:  0.9842
Epoch:  25 Accuracy:  0.9854
Epoch:  26 Accuracy:  0.984
Epoch:  27 Accuracy:  0.9854
Epoch:  28 Accuracy:  0.9848
Epoch:  29 Accuracy:  0.9857
Epoch:  30 Accuracy:  0.9861
Epoch:  31 Accuracy:  0.9851
Epoch:  32 Accuracy:  0.9856
Epoch:  33 Accuracy:  0.9852
Epoch:  34 Accuracy:  0.9873
Epoch:  35 Accuracy:  0.986


In [13]:
#When finish, close the interactive session
sess.close()

# Reset the graph to the next experiments
tf.reset_default_graph()

## Use tensorboard to show the net & the training process.
    - The same previous convolutional model with the commands that need tensorboard

Based on https://www.tensorflow.org/how_tos/summaries_and_tensorboard/index.html 

In [18]:
def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/'   + name, mean)
        tf.summary.scalar('sttdev/' + name, tf.sqrt(tf.reduce_mean(tf.square(var - mean))))
        tf.summary.scalar('max/'    + name, tf.reduce_max(var))
        tf.summary.scalar('min/'    + name, tf.reduce_min(var))
        tf.summary.histogram(name, var)


In [19]:
# Start interactive session
sess = tf.InteractiveSession()


# Convolutional model
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y = tf.placeholder(tf.int64, shape=[None, ])

image = tf.reshape(x,[-1,28,28,1])

with tf.name_scope('Conv1') as scope:
    conv1 = layers.Conv2D(20, (5,5))(image)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    variable_summaries(pool1, "pool1_summary")

with tf.name_scope('Conv2') as scope:
    conv2 = layers.Conv2D(20, (5,5))(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    variable_summaries(pool2, "pool2_summary")

with tf.name_scope('Dense') as scope:
    flat = layers.Flatten(name='Flat_image')(pool2)
    # Tensorflow Dense layer
    W_dense = tf.Variable(tf.truncated_normal([320, 500], stddev=0.1), name='W_dense')
    variable_summaries(W_dense, "W_dense_summary") # Summaries of the layer weigths
    b_dense = tf.Variable(tf.constant(0.1, shape=[500]), name='b_dense')
    variable_summaries(b_dense, "b_dense_summary") # Summaries of the layer weigths
    dense1 = tf.nn.relu(tf.matmul(flat, W_dense) + b_dense)
    variable_summaries(dense1, "dense1_summary") # Summaries of the layer output

    output = layers.Dense(10, activation='linear', name='Dense_output')(dense1)

# Define the Tensorflow Loss
#with tf.name_scope('loss') as scope:
cross_entropy = tf.losses.sparse_softmax_cross_entropy(y, output)
loss_summary = tf.summary.scalar("loss", cross_entropy)
    
# Define the Tensorflow Train function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Create an Accuracy metric to evaluate in test
#with tf.name_scope('acc') as scope:
y_pred = tf.nn.softmax(output)
correct_prediction = tf.equal(y, tf.argmax(y_pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_summary = tf.summary.scalar("acc", accuracy)

# Add summaries to the graph
summaries_dir = '/tmp/tensorboard/tf'
with tf.name_scope('summaries') as scope:
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
    test_writer  = tf.summary.FileWriter(summaries_dir + '/test')

    

In [20]:
# Intialize vars
sess.run(tf.global_variables_initializer())

# Early stopping code. Stop if the current accuray is < that the min of the last 5 epochs. 
batch_size = 128
acc_test=[]
epoch=0
stop=True
while stop:
    #Train
    epoch += 1
    counter = 0
    for x_batch, y_batch in  batch_generator(X_train, y_train, batch_size=batch_size):
        train_step.run(feed_dict={x: x_batch, y: y_batch})
        
        counter += 1
        if counter%10 == 0:
            summary_str = merged.eval(feed_dict={x: x_batch, y: y_batch}) #TENSORBOARD
            train_writer.add_summary(summary_str, epoch) #TENSORBOARD
            
                
    # Test
    acc_test += [accuracy.eval(feed_dict={x: X_test, y: y_test})]
    
    summary_str = merged.eval(feed_dict={x: X_test, y: y_test}) #TENSORBOARD 
    test_writer.add_summary(summary_str, epoch) #TENSORBOARD 
      
    if epoch%1==0:
        print('Epoch: ', epoch, 'Accuracy: ', acc_test[-1])

    # Stopping criteria
    if epoch>10 and acc_test[-1] < min(acc_test[-10:-1]):
        stop=False
        print('STOP. Accuracy: ', acc_test[-1])
            

Epoch:  1 Accuracy:  0.9169
Epoch:  2 Accuracy:  0.9417
Epoch:  3 Accuracy:  0.9528
Epoch:  4 Accuracy:  0.9624
Epoch:  5 Accuracy:  0.9667
Epoch:  6 Accuracy:  0.9668
Epoch:  7 Accuracy:  0.9739
Epoch:  8 Accuracy:  0.975
Epoch:  9 Accuracy:  0.9761
Epoch:  10 Accuracy:  0.9788
Epoch:  11 Accuracy:  0.9794
Epoch:  12 Accuracy:  0.9794
Epoch:  13 Accuracy:  0.9794
Epoch:  14 Accuracy:  0.9787
Epoch:  15 Accuracy:  0.9806
Epoch:  16 Accuracy:  0.9829
Epoch:  17 Accuracy:  0.9826
Epoch:  18 Accuracy:  0.9841
Epoch:  19 Accuracy:  0.9841
Epoch:  20 Accuracy:  0.9849
Epoch:  21 Accuracy:  0.9834
Epoch:  22 Accuracy:  0.9861
Epoch:  23 Accuracy:  0.9851
Epoch:  24 Accuracy:  0.9859
Epoch:  25 Accuracy:  0.9852
Epoch:  26 Accuracy:  0.9869
Epoch:  27 Accuracy:  0.9855
Epoch:  28 Accuracy:  0.9852
Epoch:  29 Accuracy:  0.9842
Epoch:  30 Accuracy:  0.9867
Epoch:  31 Accuracy:  0.9843
Epoch:  32 Accuracy:  0.987
Epoch:  33 Accuracy:  0.9861
Epoch:  34 Accuracy:  0.9877
Epoch:  35 Accuracy:  0.9

In [21]:
sess.close()
tf.reset_default_graph()

At the end execute tensorboar with:

    cd /tmp

    tensorboard --logdir=./tensorboard

And accest to it in:

    http://<My_server_IP>:6006
